# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [ ]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [2]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_12176/1427064109.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [8]:
from langchain_core.tools import tool

# Define a tool to multiply two numbers
@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y

# Define a tool to add two numbers
@tool
def add(x: int, y: int) -> int:
    """Add two numbers together."""
    return x + y

# Define a tool to subtract two numbers
@tool
def subtract(x: float, y: float) -> float:
    """Subtract y from x."""
    return x - y

# List of available tools
tools = [multiply, add, subtract]

# Inspect each tool's metadata
for tool_instance in tools:
    print("--")
    print(f"Tool Name: {tool_instance.name}")
    print(f"Description: {tool_instance.description}")
    print(f"Arguments: {tool_instance.args}")

--
Tool Name: multiply
Description: Multiply two numbers together.
Arguments: {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
Tool Name: add
Description: Add two numbers together.
Arguments: {'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
Tool Name: subtract
Description: Subtract y from x.
Arguments: {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}


In [9]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers together.
subtract(x: float, y: float) -> float - Subtract y from x.


In [22]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
model = ChatOllama(model="llama3")

/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_12176/3399379857.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3")


In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

# Render tool descriptions
rendered_tools = ""
for t in tools:
    rendered_tools += f"- {t.name}: {t.description}\n  Arguments: {t.args}\n\n"

# System prompt with escaped curly braces
system_prompt = """\
You are an intelligent assistant that can use the following tools:

{rendered_tools}

Each tool has a name, a description, and a set of required arguments.

Your task is to read the user's request and decide which tool to use, then return a JSON object in the following format:

{{
  "name": "<tool_name>",
  "arguments": {{
    "<arg1>": <value1>,
    "<arg2>": <value2>
  }}
}}

Only return this JSON. Do not include any other explanation or text.
"""

# Build the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

# Load the model from Ollama
model = ChatOllama(model="llama3")  # or mistral, codellama, etc.

# Chain the prompt and model
chain = prompt | model

# Invoke the chain
message = chain.invoke({
    "input": "What's 10 times 3?",
    "rendered_tools": rendered_tools
})

# Print the result
print(message.content if hasattr(message, 'content') else message)

{
  "name": "multiply",
  "arguments": {
    "x": 10,
    "y": 3
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [25]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
# Invoke the chain with user input
message = chain.invoke({
    "input": "what's thirteen times 4",
    "rendered_tools": rendered_tools
})

# Print the result (which will now be parsed as JSON)
print(message)

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}


:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [26]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import Tool  # Ensure you're using the correct tool class from LangChain

# Define a TypedDict to represent the inputs for the invoke_tool function
class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""
    name: str
    arguments: Dict[str, Any]

# Define the function to invoke a tool by its name and arguments
def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """Function to invoke a tool by its name and arguments.

    Args:
        tool_call_request: A dictionary containing the name and arguments of the tool to invoke.
        config: Optional configuration for the tool invocation, like callbacks or metadata.

    Returns:
        Output from the requested tool.
    """
    # Create a mapping from tool name to tool instance
    tool_name_to_tool = {tool.name: tool for tool in tools}

    # Extract the tool name from the request
    name = tool_call_request["name"]
    
    # Get the requested tool by its name
    requested_tool = tool_name_to_tool.get(name)
    if requested_tool is None:
        raise ValueError(f"Tool '{name}' not found.")
    
    # Invoke the tool with the given arguments
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

# Example of how to use the invoke_tool function
tool_call_request = {
    "name": "multiply",  # For example, use the 'multiply' tool
    "arguments": {"x": 13, "y": 4}
}

result = invoke_tool(tool_call_request)
print(result)

52.0


Let's test this out 🧪!

In [28]:
tool_call_request = {
    "name": "multiply",  # Use the 'multiply' tool
    "arguments": {"x": 13, "y": 4}
}

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [30]:
# Create the chain with the prompt, model, JSON output parser, and invoke_tool
chain = prompt | model | JsonOutputParser() | invoke_tool

# Invoke the chain with user input
message = chain.invoke({
    "input": "what's thirteen times 4.14137281",
    "rendered_tools": rendered_tools
})

# Print the result
print(message)

53.83784653


## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [32]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt 
    | model 
    | JsonOutputParser() 
    | RunnablePassthrough.assign(output=invoke_tool)  # Pass the output to invoke_tool
)

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

Strategies to Improve Model Output:

1. Provide Few-Shot Examples
	•	Why: Few-shot learning can help guide the model by providing it with concrete examples of what the expected output should look like. This makes it more likely to generate the correct format or invoke the correct tool.
	•	How:
	•	Include multiple examples within the prompt to show the model exactly how to handle tool invocations.
	•	Example: Show it examples of different mathematical operations (e.g., addition, multiplication, etc.) with the proper JSON output format.

In [ ]:
system_prompt = """\
You are an assistant that can perform the following operations. Each tool call should be output as a JSON object with a 'name' field and an 'arguments' field.

Here are a few examples:

Example 1:
Input: "What's 5 plus 10?"
Output: {"name": "add", "arguments": {"x": 5, "y": 10}}

Example 2:
Input: "What is 4 times 5?"
Output: {"name": "multiply", "arguments": {"x": 4, "y": 5}}

Example 3:
Input: "What is 10 minus 3?"
Output: {"name": "subtract", "arguments": {"x": 10, "y": 3}}

Now, based on the user's input, please provide the correct JSON object.
"""

2. Add Error Handling
	•	Why: Even with the best setup, the model might still make mistakes, particularly when it’s not trained to handle tool calls directly. Error handling ensures that errors don’t halt the process and that the system can try to correct itself.
	•	How:
	•	Catch the exceptions: If the model returns an unexpected result or fails to properly format the output, you can catch the error and refeed the incorrect output back into the model for correction.
	•	Example of error handling:

In [ ]:
try:
    # Try to invoke the chain with the user input
    message = chain.invoke({"input": "what's thirteen times 4.14137281"})
    print(message)
except Exception as e:
    # If an error occurs, prompt the model to fix the output
    print(f"Error occurred: {str(e)}")
    correction_request = "It seems like the tool invocation output was incorrect. Please fix your output and provide the correct JSON."
    corrected_message = model.invoke({"input": correction_request})
    print(f"Corrected message: {corrected_message}")

	•	What this does: If the invoke_tool step throws an exception, the error handling section captures the error and then re-feeds the issue back to the model, asking it to correct its previous mistake.

Key Takeaways:
	•	Few-shot examples improve model accuracy and help it generate valid tool invocations.
	•	Error handling ensures the system doesn’t fail when the model makes mistakes and can self-correct.

    By preparing for potential issues and using these strategies, you can significantly improve the robustness of your system and ensure that it handles real-world situations more effectively.